## [Running Flask and FastAPI on Google Colab](https://medium.datadriveninvestor.com/flask-on-colab-825d2099d9d8)

In [5]:
!pip install fastapi nest-asyncio pyngrok uvicorn

['Collecting fastapi',
 '\x1b[?25l  Downloading https://files.pythonhosted.org/packages/4e/b9/a91a699f5c201413b3f61405dbccc29ebe5ad25945230e9cec98fdb2434c/fastapi-0.65.1-py3-none-any.whl (50kB)',
 '',
 '\x1b[K     |██████▍                         | 10kB 17.0MB/s eta 0:00:01',
 '\x1b[K     |████████████▉                   | 20kB 19.1MB/s eta 0:00:01',
 '\x1b[K     |███████████████████▎            | 30kB 10.4MB/s eta 0:00:01',
 '\x1b[K     |█████████████████████████▊      | 40kB 168kB/s eta 0:00:01',
 '\x1b[K     |████████████████████████████████| 51kB 203kB/s ',
 '\x1b[?25hRequirement already satisfied: nest-asyncio in /usr/local/lib/python3.7/dist-packages (1.5.1)',
 'Collecting pyngrok',
 '\x1b[?25l  Downloading https://files.pythonhosted.org/packages/6b/4e/a2fe095bbe17cf26424c4abcd22a0490e22d01cc628f25af5e220ddbf6f0/pyngrok-5.0.5.tar.gz (745kB)',
 '',
 '\x1b[K     |▍                               | 10kB 502kB/s eta 0:00:02',
 '\x1b[K     |▉                               | 20kB 700kB/

In [6]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn

app = FastAPI()

@app.get('/index')
async def home():
  return "Hello World"

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)


Public URL: http://32e225e77f5d.ngrok.io


INFO:     Started server process [58]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     124.32.186.1:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     124.32.186.1:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     124.32.186.1:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     124.32.186.1:0 - "GET /index HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [58]


## [Deploying Transformer Models](https://chatbotslife.com/deploying-transformer-models-1350876016f)


In [1]:
!pip install -qq transformers

     |████████████████████████████████| 2.1MB 6.4MB/s 
     |████████████████████████████████| 901kB 21.9MB/s 
     |████████████████████████████████| 3.3MB 34.5MB/s 


In [2]:
from transformers import (
    pipeline,
    GPT2LMHeadModel,
    GPT2Tokenizer
)

class NLP:
    def __init__(self):
        self.gen_model = GPT2LMHeadModel.from_pretrained('gpt2')
        self.gen_tokenizer = GPT2Tokenizer.from_pretrained('gpt2') 
         
    def generate(self, prompt="The epistemelogical limit"):
        
        inputs = self.gen_tokenizer.encode( prompt, add_special_tokens=False, return_tensors="pt")
        prompt_length = len(self.gen_tokenizer.decode(inputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
        outputs = self.gen_model.generate(inputs, max_length=200, do_sample=True, top_p=0.95, top_k=60)
        generated = prompt + self.gen_tokenizer.decode(outputs[0])[prompt_length:]
        return generated

    def sentiments(self, text: str):
        nlp = pipeline("sentiment-analysis")
        result = nlp(text)[0]
        return f"label: {result['label']}, with score: {round(result['score'], 4)}"


In [4]:
# from nlp import NLP
nlp = NLP()
print(nlp.sentiments("A bee sting is not cool"))



label: NEGATIVE, with score: 0.9998


In [8]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
#from app.nlp import NLP

class Message(BaseModel):
    input: str
    output: str = None

app = FastAPI()
nlp = NLP()

origins = [
    "http://localhost",
    "http://localhost:3000",
    "http://127.0.0.1:3000"
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["POST"],
    allow_headers=["*"],
)

@app.post("/generative/")
async def  generate(message: Message):
    message.output  = nlp.generate(prompt=message.input)
    return {"output" : message.output}

@app.post("/sentiment/")
async def sentiment_analysis(message: Message):
    message.output  = str(nlp.sentiments(message.input))
    return {"output" : message.output}

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://0f9b3fe339a5.ngrok.io


INFO:     Started server process [58]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     124.32.186.1:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     124.32.186.1:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     124.32.186.1:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     124.32.186.1:0 - "GET /openapi.json HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


INFO:     124.32.186.1:0 - "POST /generative/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [58]


```
% curl -X POST "http://0f9b3fe339a5.ngrok.io/generative/" -H "accept: application/json" -H "Content-Type: application/json" -d "{\"input\":\"Basil pasta is really nice with\"}"

{"output":"Basil pasta is really nice with the extra salad that I like the best. When I started this recipe this year, I was worried I would make too much pasta with my family but now that I'm on a budget, I'm happy.\n\nNow, the fact of the matter is I'm a fan of pasta with a spicy twist. I'm going to make it spicy this year. I'm going to make it delicious with broccoli, tomato sauce, broccoli crumbles, cheese, cheddar, feta, chives, chili, garlic, cheddar, and onions (a side.)\n\nYou want to make this pasta using your favorite recipe from these two posts, and then add it to a dish with more veggies or other ingredients on top. This is my recipe for a grilled casserole, and I've been using broccoli cauliflower and cauliflower noodles at least 3 times in my pasta routine, so this recipe is a lot easier for me than the simple, easy"}
```